In [76]:
import pandas as pd
import numpy as np
import json
import os

# custom functions
import functions

# bert and pytorch packages
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification

In [103]:
# load and unpack

fp = 'data/news/raw/netflix_stock_news.json'

with open(fp) as json_file:
    data = json.load(json_file)
    news_df = pd.read_json(fp)
articles = functions.newsapiArticleUnpack(news_df)

recent_fp = 'data/news/raw/netflix_stock_recent.json'

# grab recent data and concat
with open(recent_fp) as json_file:
    data = json.load(json_file)
    news_df = pd.read_json(recent_fp)
new_articles = functions.newsapiArticleUnpack(news_df)

articles = pd.concat([articles, new_articles])
articles.tail()

,author,content,description,publishedAt,source,title,url,urlToImage
7,"Jon Swartz, , Jeremy C. Owens",Walt Disney Co. added more streaming subscribe...,Walt Disney Co. added more streaming subscribe...,2022-08-10T20:25:55Z,"{'id': None, 'name': 'MarketWatch'}",: Disney to raise streaming prices while launc...,https://www.marketwatch.com/story/disney-to-ra...,https://images.mktw.net/im-515943/social
8,bizpoke,Walt Disney Co. added more streaming subscribe...,Walt Disney Co. added more streaming subscribe...,2022-08-11T10:24:46Z,"{'id': None, 'name': 'Biztoc.com'}",Disney to raise streaming prices while launchi...,https://biztoc.com/p/zf4rszsz?ref=rss,https://cdn.biztoc.com/og/zf4rszsz.jpg
9,AJ Fabino,Loup Ventures is a tech centered hedge fund wi...,Loup Ventures is a tech centered hedge fund wi...,2022-08-12T20:50:57Z,"{'id': None, 'name': 'Benzinga'}",EXCLUSIVE: Gene Munster Shares Top 2023 Tech P...,https://www.benzinga.com/analyst-ratings/analy...,https://cdn.benzinga.com/files/images/story/20...
10,bizpoke,Walt Disney Co. added more streaming subscribe...,Walt Disney Co. added more streaming subscribe...,2022-08-11T10:24:46Z,"{'id': None, 'name': 'Biztoc.com'}",Disney to raise streaming prices while launchi...,https://biztoc.com/p/zf4rszsz?ref=rss,https://cdn.biztoc.com/og/zf4rszsz.jpg
11,AJ Fabino,Loup Ventures is a tech centered hedge fund wi...,Loup Ventures is a tech centered hedge fund wi...,2022-08-12T20:50:57Z,"{'id': None, 'name': 'Benzinga'}",EXCLUSIVE: Gene Munster Shares Top 2023 Tech P...,https://www.benzinga.com/analyst-ratings/analy...,https://cdn.benzinga.com/files/images/story/20...


In [104]:
# remove duplicates
articles.drop_duplicates(subset=['description'], inplace=True)

In [105]:
articles

,author,content,description,publishedAt,source,title,url,urlToImage
0,newsfeedback@fool.com (Parkev Tatevosian),Netflix(NFLX -1.21%) said that the latest seas...,"""Stranger Things"" has captured over 1 billion ...",2022-07-10T10:05:00Z,"{'id': None, 'name': 'Motley Fool'}",Why Is Everyone Talking About Netflix Stock?,https://www.fool.com/investing/2022/07/10/why-...,https://g.foolcdn.com/editorial/images/688731/...
1,newsfeedback@fool.com (Demitri Kalogeropoulos),What happened\r\nNetflix(NFLX -4.09%) sharehol...,Investors are bracing for more bad news from t...,2022-07-11T16:20:06Z,"{'id': None, 'name': 'Motley Fool'}",Why Netflix Stock Fell 4% Today,https://www.fool.com/investing/2022/07/11/why-...,https://g.foolcdn.com/editorial/images/689459/...
4,newsfeedback@fool.com (Anders Bylund),Media-streaming veteran Netflix(NFLX -1.87%) i...,The digital video maven will report earnings n...,2022-07-12T14:05:22Z,"{'id': None, 'name': 'Motley Fool'}",Should You Buy Netflix Stock Before Next Week'...,https://www.fool.com/investing/2022/07/12/shou...,https://g.foolcdn.com/editorial/images/689628/...
5,The Value Pendulum,stockcam\r\nElevator Pitch\r\n My investment r...,Netflix has been hit by increased subscriber c...,2022-07-12T14:09:47Z,"{'id': None, 'name': 'Seeking Alpha'}",Is Netflix Stock A Sell Before Upcoming Earnin...,https://seekingalpha.com/article/4522892-is-ne...,https://static.seekingalpha.com/cdn/s3/uploads...
8,None,"NEW YORK, July 13, 2022 /PRNewswire/ -- Invest...","NEW YORK, July 13, 2022 /PRNewswire/ -- Invest...",2022-07-13T14:55:00Z,"{'id': None, 'name': 'PRNewswire'}",Thinking about trading options or stock in Adv...,https://www.prnewswire.com/news-releases/think...,https://mma.prnewswire.com/media/1333368/Inves...
...,...,...,...,...,...,...,...,...
2,newsfeedback@fool.com (Dani Cook),Increased streaming competition has hurt few c...,The company is directly responsible for the ri...,2022-08-09T11:32:00Z,"{'id': None, 'name': 'Motley Fool'}","Netflix Stock Slump: Despite Recent Losses, Th...",https://www.fool.com/investing/2022/08/09/netf...,https://g.foolcdn.com/editorial/images/694108/...
3,newsfeedback@fool.com (Tom Wilton),Warner Bros. Discovery(WBD -3.91%) has announc...,Warner Bros. Discovery is putting everything i...,2022-08-09T11:18:00Z,"{'id': None, 'name': 'Motley Fool'}",Will This Mega Streaming Stock Spell Trouble f...,https://www.fool.com/investing/2022/08/09/will...,https://g.foolcdn.com/editorial/images/694871/...
6,"Jon Swartz, , Jeremy C. Owens",Walt Disney Co. added more streaming subscribe...,Walt Disney Co. added more streaming subscribe...,2022-08-10T20:25:55Z,"{'id': None, 'name': 'MarketWatch'}",: Disney to raise streaming prices while launc...,https://www.marketwatch.com/story/disney-to-ra...,https://images.mktw.net/im-515943/social
8,bizpoke,Walt Disney Co. added more streaming subscribe...,Walt Disney Co. added more streaming subscribe...,2022-08-11T10:24:46Z,"{'id': None, 'name': 'Biztoc.com'}",Disney to raise streaming prices while launchi...,https://biztoc.com/p/zf4rszsz?ref=rss,https://cdn.biztoc.com/og/zf4rszsz.jpg


In [106]:
model_fp = 'models/classifier_model/finbert-sentiment'
text_fp = 'data/news/to_classify/content_processed.txt'

finbert = AutoModelForSequenceClassification.from_pretrained(model_fp,num_labels=3,cache_dir=None)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [107]:
# prep data for classification
text_dict = {}

for column in articles[['title','description']]:
    text_dict[column] = list(articles[column])

In [108]:
# classify data using pytorch get label probabilities

cat_dict = {}
neutral_dict = {}
positive_dict = {}
negative_dict = {}

labels = {0:'neutral', 1:'positive',2:'negative'}

for column in text_dict.keys():
    cat_list = []
    neutral = []
    positive = []
    negative = []
    for sentence in text_dict[column]:
        # send and receive data
        inputs = tokenizer(sentence, return_tensors="pt", padding=True)
        outputs = finbert(**inputs)[0]
        # only take the max prob to categorize
        sentiment = labels[np.argmax(outputs.detach().numpy())]
        print(sentence, '#####', sentiment)
        # store values
        cat_list.append(sentiment)
        neutral.append(outputs.detach().numpy()[0][0])
        positive.append(outputs.detach().numpy()[0][1])
        negative.append(outputs.detach().numpy()[0][2])

    cat_dict[column] = cat_list
    neutral_dict[column] = neutral
    positive_dict[column] = positive
    negative_dict[column] = negative

Why Is Everyone Talking About Netflix Stock? ##### negative
Why Netflix Stock Fell 4% Today ##### positive
Should You Buy Netflix Stock Before Next Week's Earnings Report? ##### negative
Is Netflix Stock A Sell Before Upcoming Earnings? Consider Short-Term Vs. Long-Term Outlook ##### negative
Thinking about trading options or stock in Advanced Micro Devices, Exxon Mobil, General Mills, Amazon, or Netflix? ##### negative
Is Netflix Stock Undervalued? ##### positive
Is Netflix Stock a Buy Before Earnings? ##### negative
Here's Why Netflix Stock Could Soar Next Week ##### negative
Better Streaming Stock: Disney vs Netflix ##### neutral
This Key Metric Could Make or Break Netflix Stock ##### negative
Better FAANG Stock Right Now: Alphabet or Netflix? ##### negative
Is Netflix Stock a Buy After Its Deal With Microsoft? ##### negative
Netflix Stock Has a Lot to Prove This Week ##### negative
Tesla, Netflix earnings, GameStop stock split, existing home sales top week ahead - Fox Business ####

In [109]:
# store values in df
for column in cat_dict.keys():
    articles[str(column)+"_sentiment_cat"] = cat_dict[column]
for column in neutral_dict.keys():
    articles[str(column)+"_sentiment_neutral"] = neutral_dict[column]
for column in positive_dict.keys():
    articles[str(column)+"_sentiment_positive"] = positive_dict[column]
for column in negative_dict.keys():
    articles[str(column)+"_sentiment_negative"] = negative_dict[column]
articles

,author,content,description,publishedAt,source,title,url,urlToImage,title_sentiment_cat,description_sentiment_cat,title_sentiment_neutral,description_sentiment_neutral,title_sentiment_positive,description_sentiment_positive,title_sentiment_negative,description_sentiment_negative
0,newsfeedback@fool.com (Parkev Tatevosian),Netflix(NFLX -1.21%) said that the latest seas...,"""Stranger Things"" has captured over 1 billion ...",2022-07-10T10:05:00Z,"{'id': None, 'name': 'Motley Fool'}",Why Is Everyone Talking About Netflix Stock?,https://www.fool.com/investing/2022/07/10/why-...,https://g.foolcdn.com/editorial/images/688731/...,negative,negative,-1.391915,0.311413,-0.004960,-2.104541,1.082948,1.902008
1,newsfeedback@fool.com (Demitri Kalogeropoulos),What happened\r\nNetflix(NFLX -4.09%) sharehol...,Investors are bracing for more bad news from t...,2022-07-11T16:20:06Z,"{'id': None, 'name': 'Motley Fool'}",Why Netflix Stock Fell 4% Today,https://www.fool.com/investing/2022/07/11/why-...,https://g.foolcdn.com/editorial/images/689459/...,positive,positive,-1.547374,-1.874073,2.198045,1.932919,-1.179389,-0.676670
4,newsfeedback@fool.com (Anders Bylund),Media-streaming veteran Netflix(NFLX -1.87%) i...,The digital video maven will report earnings n...,2022-07-12T14:05:22Z,"{'id': None, 'name': 'Motley Fool'}",Should You Buy Netflix Stock Before Next Week'...,https://www.fool.com/investing/2022/07/12/shou...,https://g.foolcdn.com/editorial/images/689628/...,negative,negative,-0.500017,-1.011215,-2.114805,-1.244079,2.427392,2.570103
5,The Value Pendulum,stockcam\r\nElevator Pitch\r\n My investment r...,Netflix has been hit by increased subscriber c...,2022-07-12T14:09:47Z,"{'id': None, 'name': 'Seeking Alpha'}",Is Netflix Stock A Sell Before Upcoming Earnin...,https://seekingalpha.com/article/4522892-is-ne...,https://static.seekingalpha.com/cdn/s3/uploads...,negative,positive,-0.821234,-1.924304,-1.093194,2.227238,1.604493,-1.104882
8,None,"NEW YORK, July 13, 2022 /PRNewswire/ -- Invest...","NEW YORK, July 13, 2022 /PRNewswire/ -- Invest...",2022-07-13T14:55:00Z,"{'id': None, 'name': 'PRNewswire'}",Thinking about trading options or stock in Adv...,https://www.prnewswire.com/news-releases/think...,https://mma.prnewswire.com/media/1333368/Inves...,negative,negative,-1.115195,-0.729644,-1.108096,-1.705350,2.584871,2.411700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,newsfeedback@fool.com (Dani Cook),Increased streaming competition has hurt few c...,The company is directly responsible for the ri...,2022-08-09T11:32:00Z,"{'id': None, 'name': 'Motley Fool'}","Netflix Stock Slump: Despite Recent Losses, Th...",https://www.fool.com/investing/2022/08/09/netf...,https://g.foolcdn.com/editorial/images/694108/...,neutral,negative,0.105902,-0.128577,-0.484353,-2.652559,-0.456198,2.842136
3,newsfeedback@fool.com (Tom Wilton),Warner Bros. Discovery(WBD -3.91%) has announc...,Warner Bros. Discovery is putting everything i...,2022-08-09T11:18:00Z,"{'id': None, 'name': 'Motley Fool'}",Will This Mega Streaming Stock Spell Trouble f...,https://www.fool.com/investing/2022/08/09/will...,https://g.foolcdn.com/editorial/images/694871/...,positive,negative,-1.985451,-0.088200,2.093941,-2.596299,-0.709575,2.672714
6,"Jon Swartz, , Jeremy C. Owens",Walt Disney Co. added more streaming subscribe...,Walt Disney Co. added more streaming subscribe...,2022-08-10T20:25:55Z,"{'id': None, 'name': 'MarketWatch'}",: Disney to raise streaming prices while launc...,https://www.marketwatch.com/story/disney-to-ra...,https://images.mktw.net/im-515943/social,neutral,neutral,2.354949,2.565963,-1.988699,-2.013045,-0.139721,-0.023205
8,bizpoke,Walt Disney Co. added more streaming subscribe...,Walt Disney Co. added more streaming subscribe...,2022-08-11T10:24:46Z,"{'id': None, 'name': 'Biztoc.com'}",Disney to raise streaming prices while launchi...,https://biztoc.com/p/zf4rszsz?ref=rss,https://cdn.biztoc.com/og/zf4rszsz.jpg,neutral,negative,2.326309,1.139124

In [114]:
# Let's only consider the useful features of the data
out_articles = articles[['publishedAt','description_sentiment_cat', 'description_sentiment_neutral', 'description_sentiment_positive', 'description_sentiment_negative']]

# convert to datetime
out_articles['publishedAt'] = pd.to_datetime(out_articles['publishedAt'])

# group by and take the mean
daily_news = out_articles.groupby([out_articles['publishedAt'].dt.date]).mean()

# turn into categorical
sentiment_list = []

for row in daily_news.itertuples():
    predictions = row[1:]
    sentiment = labels[np.argmax(predictions)]
    sentiment_list.append(sentiment)

daily_news['news_daily_sentiment'] = sentiment_list

C:\Users\lapen\AppData\Local\Temp/ipykernel_21088/902232580.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_articles['publishedAt'] = pd.to_datetime(out_articles['publishedAt'])


In [115]:
daily_news

,description_sentiment_neutral,description_sentiment_positive,description_sentiment_negative,news_daily_sentiment
publishedAt,,,,
2022-07-10,0.311413,-2.104541,1.902008,negative
2022-07-11,-1.874073,1.932919,-0.676670,positive
2022-07-12,-1.467759,0.491579,0.732611,negative
2022-07-13,-0.729644,-1.705350,2.411700,negative
2022-07-14,0.618926,-1.880787,0.899333,negative
2022-07-15,0.556324,-0.761079,0.059699,neutral
2022-07-17,1.934240,-2.715606,0.936026,neutral
2022-07-18,-0.981156,0.262897,0.054096,positive
2022-07-19,0.106797,-0.265081,-0.120844,neutral


In [116]:
# export non-aggregrated
out_articles.to_csv('data/news/classified/news_sentiment.csv')

In [117]:
# export daily
daily_news.to_csv('data/news/classified/news_mean_sentiment.csv')